# Preprocessing

In [4]:
import pandas as pd
import numpy as np
import re
from pre_processing import *
from textblob import TextBlob

import os

In [5]:
re

<module 're' from '/Users/robinjaccard/opt/anaconda3/lib/python3.8/re.py'>

In [6]:
dfs = []
for r, d, f in os.walk(os.getcwd()):
    for file in f:
        if 'withheldtweets.json' in file:
            dfs.append(pd.read_json("./censored_tweets/%s" % file, lines=True))

df_cen = pd.concat(dfs)
df_cen = df_cen.dropna(subset=['withheld_in_countries'])
df_cen.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
1,2021-02-26 10:48:35+00:00,1365252415444946945,1365252415444946944,#Balakot \nPak Army is our pride ❤️👍 https://t...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-26 10:48:35.662,no,"[0, 35]",[IN],"{'media': [{'id': 1365252409015033857, 'id_str...",0.0,NaN,NaN
2,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,no,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
3,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,retweeted,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
4,2021-02-26 10:38:57+00:00,1365249991137251328,1365249991137251328,RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ض...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:38:57.662,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:23:00 +0000 2021...,NaN
5,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,no,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
6,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,retweeted,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
7,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,no,NaN,[IN],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
8,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,retweeted,NaN,[IN],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
9,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN
10,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,retweeted,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN


In [7]:
pd.options.display.max_colwidth = 500
pd.options.display.width = 300
df_english = df_cen[df_cen['lang'] == 'en']

In [8]:
list_of_languages = df_cen['lang'].unique()
list_of_languages

array(['en', 'ur', 'ar', 'hi', 'und', 'cs', 'tr', 'in', 'de', 'da', 'es',
       'ht', 'fr', 'tl', 'hu', 'uk', 'sd', 'bn', 'my', 'nl', 'et', 'ro',
       'pa', 'fa', 'th', 'ja', 'fi', 'el', 'pt', 'it', 'zh', 'vi', 'ckb',
       'ca', 'pl', 'sv', 'lt', 'cy', 'lv', 'km', 'ko', 'mr', 'ru', 'lo',
       'ps', 'sl', 'ta', 'no', 'is', 'eu'], dtype=object)

In [9]:
def create_csv_traduction(dataframe):
    list_en=[]
    texts = list(dataframe['text'])
    langs = list(dataframe['lang'])
    for s in dataframe.shape[0]:
        if langs[s] != 'und':
            result = translator.translate(texts[s], "english", langs[s])
            list_en.append(result)
    
    #df_new = dataframe[dataframe['lang'] != 'und']
    #df_new['Translation'] = list_en
    return pd.DataFrame(list_en, columns = 'translation')
    #df_new.to_csv(".\EnglishTranslation.csv")

In [10]:
df_test = df_cen.head(100)
df_gogogo = create_csv_traduction(df_test)
print('done')

TypeError: 'int' object is not iterable

In [ ]:
def lower_text(df_text):
    return df_text.apply(lambda x: x.lower())

In [ ]:
df_test = df_cen.head(10)

In [ ]:
lower_test = lower_text(df_cen['text'])
lower_test

In [ ]:
df_cen[df_cen['lang']=='und']

In [ ]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [ ]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

In [ ]:
def spelling_correction(tweets):
    textBlb = TextBlob(tweets)           
    return textBlb.correct()

In [ ]:
def remove_char(tweets):
    return re.sub(r"[^a-zA-Z0-9# ]", " ", tweets)

stop_words = set(stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()

def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    

def clean_tweets_after_trad(df_text):
    clean_tweets = df_text.apply(lambda x: remove_char(x))
    clean_tweets = pd.DataFrame([ele for ele in clean_tweets if ele != ''], columns =['text'])['text']
    clean_tweets = clean_tweets.apply(lambda x: x.lower())
    #clean_tweets = clean_tweets.apply(lambda x: spelling_correction(x))
    clean_tweets = clean_tweets.apply(lambda x : lemmatize_text(x))
    clean_tweets = clean_tweets.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]

### Pour Robin ###

In [11]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def remove_char(tweets):
    return re.sub(r"[^a-zA-Z0-9# ]", " ", tweets)

stop_words = set(stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()

def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    

def clean_tweets_after_trad(df_text):
    clean_tweets = df_text.apply(lambda x: remove_char(x))
    clean_tweets = pd.DataFrame([ele for ele in clean_tweets if ele != ''], columns =['text'])['text']
    clean_tweets = clean_tweets.apply(lambda x: x.lower())
    clean_tweets = clean_tweets.apply(lambda x : lemmatize_text(x))
    clean_tweets = clean_tweets.apply(lambda x: [item for item in x if item not in stop_words])

### Fin ###

In [13]:
import re

In [14]:
df_txt_clean = clean_tweets(df_en['text'])
df_txt_clean 

NameError: name 'df_en' is not defined

In [15]:
pd.DataFrame(df_txt_clean)

NameError: name 'df_txt_clean' is not defined

In [16]:
df_clean_after = clean_tweets_after_trad(pd.DataFrame(df_txt_clean)['text'])
df_clean_after

NameError: name 'df_txt_clean' is not defined

# BTM

In [61]:
import bitermplus as btm
import numpy as np
import pandas as pd

In [62]:
#my_df = pd.read_csv('./out_clean.csv')
my_df = pd.read_csv('data/to_be_clustered.csv.gz', compression="gzip")

In [63]:
my_df['clean'].apply(lambda x: str(x))
in_df = my_df[my_df['whcs'] == "France"]
in_df = in_df['clean'].dropna()
texts = in_df.str.strip().tolist()

In [64]:
# PREPROCESSING
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()
# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))
# Generating biterms
biterms = btm.get_biterms(docs_vec)

In [141]:
# INITIALIZING AND RUNNING MODEL
#model = btm.BTM(
#    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.01)
# 1000, 8, 20, 10/8, 0,1
model = btm.BTM(
    X, vocabulary, seed=1000, T=14, M=20, alpha=0.0000001, beta=1)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

100%|██████████████████████████████████████| 900/900 [00:00<00:00, 76155.46it/s]


In [142]:
# METRICS
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)

In [143]:
perplexity

1690.4946391552608

In [144]:
coherence

array([-404.7659609 , -525.53980897, -149.78715713, -149.78715713,
       -408.18676594, -455.25539157, -113.17284947, -448.41461985,
       -149.78715713, -149.78715713, -442.32133113, -122.20608224,
         12.44954718, -352.47143298])

In [145]:
p_zd

array([[1.06584605e-02, 1.99347051e-01, 7.37404624e-13, ...,
        3.08804406e-03, 5.26521302e-03, 6.45287780e-03],
       [3.12527155e-02, 7.68696192e-01, 1.09165738e-12, ...,
        3.99293533e-03, 3.40799693e-03, 8.01507073e-03],
       [8.15735265e-03, 9.40009085e-01, 1.84487538e-13, ...,
        1.22331443e-03, 5.86392106e-04, 8.25481000e-03],
       ...,
       [1.94339063e-02, 7.57982537e-01, 6.72439448e-13, ...,
        3.86848822e-03, 1.28638713e-03, 2.79507724e-03],
       [1.67547310e-02, 7.61056628e-01, 7.51236642e-13, ...,
        4.39769270e-03, 1.43712739e-03, 4.79204675e-03],
       [1.40318605e-02, 2.57851966e-01, 2.39437854e-12, ...,
        3.41467667e-02, 4.58048344e-03, 2.70811355e-02]])

In [146]:
top_words = btm.get_top_topic_words(model, words_num=10)
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13
0,always,white,zomato,zomato,woman,muslim,courage,great,zomato,zomato,amp,nigga,wilder,cpac
1,anyone,people,fight,fight,amp,year,union,even,fight,fight,left,lil,maharashtra,trump
2,know,get,fire,fire,sharia,new,major,soral,fire,fire,biden,poverty,shivgaan,must
3,communism,black,fine,fine,let,terrorist,catherine,one,fine,fine,attack,percent,birth,lincoln
4,national,say,find,find,hijab,migrant,something,someone,find,find,wing,shame,maharaj,president
5,want,anti,finally,finally,islamic,twitter,anyrtgingh,big,finally,finally,video,andrew,round,republican
6,social,one,final,final,cair,attack,salutation,world,final,final,warn,release,shivaji,project
7,firearm,go,filth,filth,one,france,infect,reset,filth,filth,border,bail,umbrella,show
8,come,like,film,film,state,illegal,visit,play,film,film,break,rise,dutch,yes
9,major,want,file,file,omar,school,music,alain,file,file,germany,hut,across,national


### Topics 
We can give categories we used for labelling to these clusters
1. 'white/black - racism', 'women' 0
1. 'jew - antisemtism' 2
1. 'tweeter acccounts', 'communism - against bankers , against state  prder ' 3
1. 'immigration' 4
1. 'farright', 'Alain Soral', 'communism - against bankers , against state  prder ' 5
1. 'islam', 'terrorism' 6
1. 'american elections' 7





1. 'homophobia'

1. Other

Don't have 
- 'India - Pakistan'
- 'coronavirus - vaccination'
- 'fantasy sex play'
- 'homophobia'

In [147]:
# RESULTS VISUALIZATION
# You need to install tmplot first
import tmplot as tmp
tmp.report(model=model, docs=texts,  show_headers = True,show_docs= False,show_topics= False, height=500, width= 500)

/Users/robinjaccard/opt/anaconda3/lib/python3.8/site-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
/Users/robinjaccard/opt/anaconda3/lib/python3.8/site-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
/Users/robinjaccard/opt/anaconda3/lib/python3.8/site-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
/Users/robinjaccard/opt/anaconda3/lib/python3.8/site-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(


In [148]:
test=model.labels_
test

array([ 5,  1,  1,  1,  5,  1,  1,  4,  1,  1,  1,  1,  0, 10,  5, 10,  1,
        1, 10,  1,  1,  5,  7,  0,  0,  1,  0,  1, 10,  4,  7,  1,  1,  1,
        1,  7,  5,  5, 10,  1,  7,  7, 10,  4, 10,  0,  1, 10,  7,  5, 10,
        1,  1, 13,  1, 10,  1, 10,  5,  1,  1,  1,  7,  1, 10,  0,  1,  1,
        1,  5,  5,  5,  7,  1,  1,  5,  1,  5,  1,  1, 11,  1,  7,  5,  4,
        1,  5,  1,  1,  7,  1, 10, 11,  4,  0,  0, 10, 10, 10,  1,  0,  7,
        1,  1,  7,  1,  1,  1,  1,  1, 10, 10,  1,  1, 10,  0,  1,  7,  1,
       13,  1,  5,  1, 10,  7,  5,  1,  1,  7,  1,  4,  5,  1,  1,  7,  1,
        0,  4,  0, 11,  0, 10,  0,  1,  1,  7,  1,  1,  1,  1,  1,  5,  1,
       13, 10,  1,  1,  1,  1,  0,  5,  1,  1,  1,  1,  0,  0, 13, 13, 10,
        7,  1,  1,  1, 10,  1, 10,  4, 10,  7, 10,  1,  1,  1, 10,  7, 13,
        7,  5,  1,  7,  7,  7,  1, 10,  7,  0,  1,  1,  1, 13,  0,  1,  1,
        1,  1, 13,  1,  4,  5, 10,  1,  1,  5,  5,  1,  1, 12,  1,  1, 10,
       10,  5, 10,  4,  1

In [149]:
model.matrix_docs_topics_[0]

array([1.06584605e-02, 1.99347051e-01, 7.37404624e-13, 7.37404624e-13,
       1.11315182e-01, 4.15532431e-01, 4.09569377e-04, 7.08484080e-03,
       7.37404624e-13, 7.37404624e-13, 2.40846331e-01, 3.08804406e-03,
       5.26521302e-03, 6.45287780e-03])

## Metric

In [150]:
from gensim.models.coherencemodel import CoherenceModel
import gensim

In [151]:
# cast tweets to numpy array
docs = my_df.clean.apply(lambda x: str(x).split()).to_numpy()

# create dictionary of all words in all documents
dictionary = gensim.corpora.Dictionary(docs)

In [153]:
# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

In [154]:
top_words = btm.get_top_topic_words(model, words_num=10)

In [155]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13
0,always,white,zomato,zomato,woman,muslim,courage,great,zomato,zomato,amp,nigga,wilder,cpac
1,anyone,people,fight,fight,amp,year,union,even,fight,fight,left,lil,maharashtra,trump
2,know,get,fire,fire,sharia,new,major,soral,fire,fire,biden,poverty,shivgaan,must
3,communism,black,fine,fine,let,terrorist,catherine,one,fine,fine,attack,percent,birth,lincoln
4,national,say,find,find,hijab,migrant,something,someone,find,find,wing,shame,maharaj,president
5,want,anti,finally,finally,islamic,twitter,anyrtgingh,big,finally,finally,video,andrew,round,republican
6,social,one,final,final,cair,attack,salutation,world,final,final,warn,release,shivaji,project
7,firearm,go,filth,filth,one,france,infect,reset,filth,filth,border,bail,umbrella,show
8,come,like,film,film,state,illegal,visit,play,film,film,break,rise,dutch,yes
9,major,want,file,file,omar,school,music,alain,file,file,germany,hut,across,national


In [156]:
top_words.T.to_numpy()

array([['always', 'anyone', 'know', 'communism', 'national', 'want',
        'social', 'firearm', 'come', 'major'],
       ['white', 'people', 'get', 'black', 'say', 'anti', 'one', 'go',
        'like', 'want'],
       ['zomato', 'fight', 'fire', 'fine', 'find', 'finally', 'final',
        'filth', 'film', 'file'],
       ['zomato', 'fight', 'fire', 'fine', 'find', 'finally', 'final',
        'filth', 'film', 'file'],
       ['woman', 'amp', 'sharia', 'let', 'hijab', 'islamic', 'cair',
        'one', 'state', 'omar'],
       ['muslim', 'year', 'new', 'terrorist', 'migrant', 'twitter',
        'attack', 'france', 'illegal', 'school'],
       ['courage', 'union', 'major', 'catherine', 'something',
        'anyrtgingh', 'salutation', 'infect', 'visit', 'music'],
       ['great', 'even', 'soral', 'one', 'someone', 'big', 'world',
        'reset', 'play', 'alain'],
       ['zomato', 'fight', 'fire', 'fine', 'find', 'finally', 'final',
        'filth', 'film', 'file'],
       ['zomato', 'fig

In [157]:
cm_gsdmm = CoherenceModel(topics=top_words.T.to_numpy(), 
                          dictionary=dictionary, 
                          corpus=bow_corpus,
                          texts=docs, 
                          coherence='c_v')

In [158]:
coherence = cm_gsdmm.get_coherence()  # get coherence value

In [159]:
coherence

0.44762523099956014